In [1]:
import pandas as pd
import os, json
import time
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import random

In [35]:
# !pip install pydicom
# !pip install pylidc

In [2]:
import pydicom
import pylidc as pl
from pylidc.utils import consensus

In [3]:
myProjectID = "w210-capstone-2021"
# BigQuery
from google.cloud import bigquery
bq = bigquery.Client()
bigquery_client = bigquery.Client(project='w210-capstone-2021')

In [4]:
extract_metadata = """
WITH
  all_lidc_ct_series AS (
  SELECT
    DISTINCT(SeriesInstanceUID),
    StudyInstanceUID,  
    PatientID,
    SliceThickness,
    ARRAY_TO_STRING(PixelSpacing,"/") as pixelspa 
  FROM
    `canceridc-data.idc_views.dicom_all`
  WHERE
    Modality = "CT"
    AND collection_id = "lidc_idri"
    AND cast(SliceThickness as decimal) < 3.0
    )
SELECT PatientID,SliceThickness,pixelspa,StudyInstanceUID,SeriesInstanceUID FROM
  all_lidc_ct_series
ORDER BY
  PatientID
  """
patientList = bq.query(extract_metadata).to_dataframe() #Get a dataframe from the query data
print("dataset shape",patientList.shape)
patientList.head() #Take a peek at the data

dataset shape (897, 5)


,PatientID,SliceThickness,pixelspa,StudyInstanceUID,SeriesInstanceUID
0,LIDC-IDRI-0001,2.500000,0.703125/0.703125,1.3.6.1.4.1.14519.5.2.1.6279.6001.298806137288...,1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636...
1,LIDC-IDRI-0002,1.250000,0.681641/0.681641,1.3.6.1.4.1.14519.5.2.1.6279.6001.490157381160...,1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417...
2,LIDC-IDRI-0003,2.500000,0.820312/0.820312,1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...
3,LIDC-IDRI-0004,1.250000,0.822266/0.822266,1.3.6.1.4.1.14519.5.2.1.6279.6001.191425307197...,1.3.6.1.4.1.14519.5.2.1.6279.6001.323541312620...
4,LIDC-IDRI-0005,2.500000,0.664062/0.664062,1.3.6.1.4.1.14519.5.2.1.6279.6001.190188259083...,1.3.6.1.4.1.14519.5.2.1.6279.6001.129007566048...


In [5]:
base_gs_uri = 'gs://idc-tcia-lidc-idri/dicom/'
patientList['image_link'] = base_gs_uri + patientList['StudyInstanceUID'] + '/' + patientList['SeriesInstanceUID']
# patientList['image_link'][5:10].to_csv("gcs_paths_3.txt",header=False, index=False)
# !head gcs_paths_3.txt

In [12]:
# !mkdir downloaded_cohort_3
# !cat gcs_paths_3.txt | gsutil -u $myProjectID -m cp -Ir ./downloaded_cohort_3

In [13]:
# !git clone https://github.com/pieper/dicomsort.git
# !pip install pydicom
# !python dicomsort/dicomsort.py --help

In [14]:
# !python dicomsort/dicomsort.py -u downloaded_cohort_3 cohort_sorted_3/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

In [44]:
patientList['PatientID']

0      LIDC-IDRI-0001
1      LIDC-IDRI-0002
2      LIDC-IDRI-0003
3      LIDC-IDRI-0004
4      LIDC-IDRI-0005
            ...      
892    LIDC-IDRI-1007
893    LIDC-IDRI-1009
894    LIDC-IDRI-1010
895    LIDC-IDRI-1011
896    LIDC-IDRI-1012
Name: PatientID, Length: 897, dtype: object

In [16]:
# f = open ('.pylidcrc','w')            #For GCP
# # f = open ('/root/.pylidcrc','w')    #For Colab
# f.write('[dicom]'+'\n')
# f.write('path =' + data_folder +'\n')
# f.write('warn = True')
# f.close()

In [17]:
# from skimage.transform import resize
# from skimage import img_as_bool

# for pa in range(len(patient_list)):
#     print ("Current patient ID =", patient_list[pa])
    
#     scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_(patient_list))
#     nodules_annotation = scan[pa].cluster_annotations()
#     vol = scan[pa].to_volume()
#     images = []
#     for i in range(vol.shape[2]):
#         images.append(vol[:,:,i].reshape(1,vol[:,:,i].shape[0],vol[:,:,i].shape[1]))
#     images = np.vstack(images)
#     images = resize(images, (images.shape[0],256,256))
#     np.save('data_npfiles/'+patient_list[pa]+'_image.npy', images)
#     # CT_image_data.append(vol)
    
#     cmask_CT = []
#     cbbox_CT = []
#     masks_CT = []
#     for nodule_idx, nodule in enumerate(nodules_annotation):
#         cmask, cbbox, masks = consensus(nodule)
#         cmask_CT.append(cmask)
#         cbbox_CT.append(cbbox)
#         masks_CT.append(masks)   

#     CT_mask = np.zeros_like(vol)    
#     nodule_num = len(cmask_CT)
    
#     for i in range(nodule_num):
#         cmask = cmask_CT[i]
#         cbbox = cbbox_CT[i]
#         masks = masks_CT[i]    
#         CT_mask[cbbox] += cmask
#     masks = []
#     for i in range(CT_mask.shape[2]):
#         masks.append(CT_mask[:,:,i].reshape(1,CT_mask[:,:,i].shape[0],CT_mask[:,:,i].shape[1]))
#     masks = (np.vstack(masks))
#     masks = img_as_bool(resize(masks, (masks.shape[0],256,256)))
#     np.save('data_npfiles/'+patient_list[pa]+'_mask.npy', masks)
    
#     # CT_mask_data.append(CT_mask)

In [18]:
# from PIL import Image
# # im = Image.fromarray(A)
# # im.save("your_file.jpeg")

# scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_(['LIDC-IDRI-0005']))
# nodules_annotation = scan[0].cluster_annotations()
# vol = scan[0].to_volume()
# image = vol[:,:,100]
# rescaled = (255.0 / image.max() * (image - image.min())).astype(np.uint8)
# im = Image.fromarray(rescaled).convert('RGB')
# # plt.imshow(vol[:,:,0])
# im.save("your_file.jpg")
# plt.imshow(im)

In [ ]:
patients_done = []
# !rm -r data_npfiles/image/img
# !mkdir data_npfiles/image/img

# !rm -r data_npfiles/mask/img
# !mkdir data_npfiles/mask/img

# !rm -r data_npfiles/val_image/img
# !mkdir data_npfiles/val_image/img

# !rm -r data_npfiles/val_mask/img
# !mkdir data_npfiles/val_mask/img

from PIL import Image

for i in range(800,900,2): #patientList.shape[0]
    patientList['image_link'][i:i+2].to_csv("gcs_paths_3.txt",header=False, index=False)

    !rm -r downloaded_cohort_3
    !rm -r cohort_sorted_3

    !mkdir downloaded_cohort_3
    !cat gcs_paths_3.txt | gsutil -u $myProjectID -m cp -Ir ./downloaded_cohort_3
    !python dicomsort/dicomsort.py -u downloaded_cohort_3 cohort_sorted_3/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm

    data_folder = os.getcwd() + '/cohort_sorted_3/'
    data_folder

    f = open ('.pylidcrc','w')            #For GCP
    # f = open ('/root/.pylidcrc','w')    #For Colab
    f.write('[dicom]'+'\n')
    f.write('path =' + data_folder +'\n')
    f.write('warn = True')
    f.close()

    patient_list = os.listdir(data_folder)
    print(patient_list)
    from skimage.transform import resize
    from skimage import img_as_bool

    for pa in range(len(patient_list)):
        print ("Current patient ID =", patient_list[pa])
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id.in_(patient_list))
        nodules_annotation = scan[pa].cluster_annotations()
        vol = scan[pa].to_volume()
        # images = []
        for i in range(vol.shape[2]):
            image = vol[:,:,i]
            # image = resize(image, (256,256))
            rescaled = (255.0 / image.max() * (image - image.min())).astype(np.uint8)
            im = Image.fromarray(rescaled)
            im.save('data_npfiles/image/img/'+patient_list[pa]+'_'+str(i)+'.tif')
            
            # images.append(vol[:,:,i].reshape(1,vol[:,:,i].shape[0],vol[:,:,i].shape[1]))
            # np.save('data_npfiles/image/img/'+patient_list[pa]+'_image_'+str(i)+'.npy', image)
        # images = np.vstack(images)
        # images = resize(images, (images.shape[0],256,256))
        # np.save('data_npfiles/'+patient_list[pa]+'_image.npy', images)
        # CT_image_data.append(vol)

        cmask_CT = []
        cbbox_CT = []
        masks_CT = []
        for nodule_idx, nodule in enumerate(nodules_annotation):
            cmask, cbbox, masks = consensus(nodule)
            cmask_CT.append(cmask)
            cbbox_CT.append(cbbox)
            masks_CT.append(masks)   

        CT_mask = np.zeros_like(vol)    
        nodule_num = len(cmask_CT)

        for i in range(nodule_num):
            cmask = cmask_CT[i]
            cbbox = cbbox_CT[i]
            masks = masks_CT[i]    
            CT_mask[cbbox] += cmask
        # masks = []
        for i in range(CT_mask.shape[2]):
            mask = CT_mask[:,:,i]
            # mask = img_as_bool(mask)
            # mask = img_as_bool(resize(mask, (256,256)))
            rescaled = (255.0 / mask.max() * (mask - mask.min())).astype(np.uint8)
            im = Image.fromarray(rescaled)
            im.save('data_npfiles/mask/img/'+patient_list[pa]+'_'+str(i)+'.tif')
            
            # masks.append(CT_mask[:,:,i].reshape(1,CT_mask[:,:,i].shape[0],CT_mask[:,:,i].shape[1]))
            # np.save('data_npfiles/mask/img/'+patient_list[pa]+'_mask_'+str(i)+'.npy', mask)
        # masks = (np.vstack(masks))
        # masks = img_as_bool(resize(masks, (masks.shape[0],256,256)))
        # np.save('data_npfiles/'+patient_list[pa]+'_mask.npy', masks)

        # CT_mask_data.append(CT_mask)
        patients_done.append(patient_list[pa])
pd.DataFrame(patients_done).to_csv('patients_done.csv')

Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.277707646521145217625235879315/1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244/1.3.6.1.4.1.14519.5.2.1.6279.6001.111588995227564299860246611072.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.277707646521145217625235879315/1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244/1.3.6.1.4.1.14519.5.2.1.6279.6001.103491071554674519221480712507.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.277707646521145217625235879315/1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244/1.3.6.1.4.1.14519.5.2.1.6279.6001.108290509405704412783309060837.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.277707646521145217625235879315/1.3.6.1.4.1.14519.5.2.1.6279.6001.254473943359963613733707320244/1.3.6.1.4.1.14519.5.2.1.6279.6001.101294384783911171930242434130.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in true_divide
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in multiply


Current patient ID = LIDC-IDRI-0894
Loading dicom files ... This may take a moment.
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.322613172334101771105940174946/1.3.6.1.4.1.14519.5.2.1.6279.6001.592821488053137951302246128864/1.3.6.1.4.1.14519.5.2.1.6279.6001.106349339561248680805248841928.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.322613172334101771105940174946/1.3.6.1.4.1.14519.5.2.1.6279.6001.592821488053137951302246128864/1.3.6.1.4.1.14519.5.2.1.6279.6001.100568138928150391677972827211.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.322613172334101771105940174946/1.3.6.1.4.1.14519.5.2.1.6279.6001.592821488053137951302246128864/1.3.6.1.4.1.14519.5.2.1.6279.6001.100470322468815562038034255463.dcm...
Copying gs://idc-tcia-lidc-idri/dicom/1.3.6.1.4.1.14519.5.2.1.6279.6001.322613172334101771105940174946/1.3.6.1.4.1.14519.5.2.1.6279.6001.592821488053137951302246128864/1.3.6.1.4.1.14519.5.2.1.6279

In [9]:
pd.DataFrame(patients_done).to_csv('patients_done.csv')

In [ ]:
files = os.listdir('data_npfiles/image/img/')
len(files)

In [32]:
files = os.listdir('data_npfiles/mask/img/')
len(files)

57900

In [4]:
# patient_list[pa]
for i in range(CT_mask.shape[2]):
    print(i,np.sum(CT_mask[:,:,i]))

NameError: name 'CT_mask' is not defined

In [12]:
images = os.listdir('data_npfiles/image/img/')
masks = os.listdir('data_npfiles/mask/img/')

missing_masks = [img for img in images if img not in masks]
len(missing_masks)

0

In [32]:
images.sort()
images[0:5]

['LIDC-IDRI-0001_0.tif',
 'LIDC-IDRI-0001_1.tif',
 'LIDC-IDRI-0001_10.tif',
 'LIDC-IDRI-0001_100.tif',
 'LIDC-IDRI-0001_101.tif']

In [33]:
masks.sort()
masks[0:5]

['LIDC-IDRI-0001_0.tif',
 'LIDC-IDRI-0001_1.tif',
 'LIDC-IDRI-0001_10.tif',
 'LIDC-IDRI-0001_100.tif',
 'LIDC-IDRI-0001_101.tif']

In [34]:
len(images) - len(masks)

523

In [9]:
missing_images = [img for img in masks if img.replace('.png','.tif',1) not in images]
len(missing_images)

0

In [7]:
missing_masks[0:5]

['LIDC-IDRI-0301_4.tif',
 'LIDC-IDRI-0301_66.tif',
 'LIDC-IDRI-0301_40.tif',
 'LIDC-IDRI-0301_30.tif',
 'LIDC-IDRI-0301_54.tif']

In [11]:
for img in missing_masks:
    file = 'data_npfiles/image/img/'+img
    # print(file)
    os.remove(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data_npfiles/image/img/LIDC-IDRI-0301_4.tif'

In [20]:
missing_masks_1 = [img for img in images if img.replace('.tif','.png',1) not in masks]
len(missing_masks_1)

165

In [22]:
missing_masks_1[0:5]

['LIDC-IDRI-0491_97.tif',
 'LIDC-IDRI-0491_11.tif',
 'LIDC-IDRI-0491_130.tif',
 'LIDC-IDRI-0491_126.tif',
 'LIDC-IDRI-0491_147.tif']

In [28]:
from PIL import Image
from skimage.transform import resize
im = Image.open('data_npfiles/mask/img/LIDC-IDRI-0001_93.png')
im.save('LIDC-IDRI-0001_0.tif')

In [30]:
masks = os.listdir('data_npfiles/mask/img/')
for mask in masks:
    old_file = 'data_npfiles/mask/img/'+mask
    new_file = old_file.replace('.png','.tif',1)
    im = Image.open(old_file)
    im.save(new_file)
    os.remove(old_file)

In [40]:
im = Image.open('data_npfiles/image/img/LIDC-IDRI-0001_93.tif')
im.save('LIDC-IDRI-0001_93_image.tif')